In [1]:
# https://github.com/abhishekshakya

import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
     


In [2]:
df = pd.read_csv('/home/sudhanshu/Desktop/resume projects/eng to hindi/my machine translator/hindi_english_parallel.csv')
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [3]:
len(df)

1561841

In [4]:
df = df.iloc[:10000,:]
df.head()
df.shape

(10000, 2)

In [5]:
data=df.values
data[0]

array(['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें',
       'Give your application an accessibility workout'], dtype=object)

In [6]:
import re

def cleanerEng(x):
    x = str(x)
    x = x.lower()
    x = re.sub(r'[^a-z0-9\s]+', '', x)  # Remove non-alphanumeric characters except spaces
    return x

def cleanerHindi(x):
    x = str(x)
    # Allow Hindi characters (Unicode range \u0900-\u097F) and spaces
    x = re.sub(r'[^\u0900-\u097F\s]+', '', x)  # Remove non-Hindi characters except spaces
    return x


In [7]:
df['english'] = df['english'].apply(func=cleanerEng)
df['hindi']= df['hindi'].apply(func= cleanerHindi)
df['english']  = df['english'].apply(func= lambda x : (str(x).split()))
df['hindi'] = df['hindi'].apply(func= lambda x : (str(x).split()))

In [8]:
df.head()

,hindi,english
0,"[अपने, अनुप्रयोग, को, पहुंचनीयता, व्यायाम, का,...","[give, your, application, an, accessibility, w..."
1,"[एक्सेर्साइसर, पहुंचनीयता, अन्वेषक]","[accerciser, accessibility, explorer]"
2,"[निचले, पटल, के, लिए, डिफोल्ट, प्लगइन, खाका]","[the, default, plugin, layout, for, the, botto..."
3,"[ऊपरी, पटल, के, लिए, डिफोल्ट, प्लगइन, खाका]","[the, default, plugin, layout, for, the, top, ..."
4,"[उन, प्लगइनों, की, सूची, जिन्हें, डिफोल्ट, रूप...","[a, list, of, plugins, that, are, disabled, by..."


In [9]:
def addTokens(x, start=False):
    if len(x) > 0:
        x.append('<eos>')
        if start:
            x.insert(0, '<sos>')
    else:
        x = ['<eos>'] if not start else ['<sos>', '<eos>']
    return list(x)


In [10]:
df['english'] = df['english'].apply(func= addTokens,start=False)
df['hindi']= df['hindi'].apply(func= addTokens,start=True)

In [11]:
data=df.values
data[0]

array([list(['<sos>', 'अपने', 'अनुप्रयोग', 'को', 'पहुंचनीयता', 'व्यायाम', 'का', 'लाभ', 'दें', '<eos>']),
       list(['give', 'your', 'application', 'an', 'accessibility', 'workout', '<eos>'])],
      dtype=object)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
     


In [13]:
data[0][0]

['<sos>',
 'अपने',
 'अनुप्रयोग',
 'को',
 'पहुंचनीयता',
 'व्यायाम',
 'का',
 'लाभ',
 'दें',
 '<eos>']

In [14]:
data[:,1].shape

(10000,)

In [15]:
import torch

class Vocab:
    def __init__(self, data, token=True):
        self.data = data
        
        if token:
            # Reserve special tokens for target language
            self.word2idx = {'<pad>': 0, '<sos>': 1, '<eos>': 2}
            self.idx2word = {0: '<pad>', 1: '<sos>', 2: '<eos>'}
            self.idx = 2  # Start from 2 since 0, 1, 2 are taken by special tokens
        else:
            # Reserve <pad> token for source language (English)
            self.word2idx = {'<pad>': 0}
            self.idx2word = {0: '<pad>'}
            self.idx = 0  # Start from 0 since only <pad> is reserved

        self.x = []
        self.max_len = 0
        self.create()
        self.vocab_size = self.idx + 1  # Final vocabulary size

    def create(self):
        # Determine max length of sentences
        for sentence in self.data:
            self.max_len = max(self.max_len, len(sentence))
            for word in sentence:
                if self.word2idx.get(word) is None:
                    self.idx += 1
                    self.word2idx[word] = self.idx
                    self.idx2word[self.idx] = word
        
        # Tokenize and pad sentences
        for sentence in self.data:
            sent = [self.word2idx[word] for word in sentence]  # Convert words to indices
            
            # Add padding tokens
            sent += [self.word2idx['<pad>']] * (self.max_len - len(sentence))
            
            self.x.append(torch.LongTensor(sent))  # Convert to LongTensor for indexing

# Example Usage
English_vocab = Vocab(data[:, 1], token=False)  # English source language
Hindi_vocab = Vocab(data[:, 0], token=True)  # Hindi target language
print(English_vocab.idx2word.keys())




dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219,

In [16]:
data[:,0]

array([list(['<sos>', 'अपने', 'अनुप्रयोग', 'को', 'पहुंचनीयता', 'व्यायाम', 'का', 'लाभ', 'दें', '<eos>']),
       list(['<sos>', 'एक्सेर्साइसर', 'पहुंचनीयता', 'अन्वेषक', '<eos>']),
       list(['<sos>', 'निचले', 'पटल', 'के', 'लिए', 'डिफोल्ट', 'प्लगइन', 'खाका', '<eos>']),
       ..., list(['<sos>', 'ईमेल', 'पताः', '<eos>']),
       list(['<sos>', 'लाइसेंसः', '<eos>']),
       list(['<sos>', 'परियोजना', 'में', 'जोड़ें', '<eos>'])],
      dtype=object)

In [17]:
for idx in Hindi_vocab.x[2]:
  print("index:", int(idx), "word:", Hindi_vocab.idx2word[int(idx)],end=' ')

print("sentence ends here")
print(len(Hindi_vocab.x[8]))

index: 1 word: <sos> index: 13 word: निचले index: 14 word: पटल index: 15 word: के index: 16 word: लिए index: 17 word: डिफोल्ट index: 18 word: प्लगइन index: 19 word: खाका index: 2 word: <eos> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> sentence ends here
40


In [18]:
for idx in English_vocab.x[2]:
  print("index:", int(idx), "word:", English_vocab.idx2word[int(idx)],end=' ')

print("sentence ends here")
print(len(English_vocab.x[8]))

index: 10 word: the index: 11 word: default index: 12 word: plugin index: 13 word: layout index: 14 word: for index: 10 word: the index: 15 word: bottom index: 16 word: panel index: 7 word: <eos> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> index: 0 word: <pad> sentence ends here
42


In [19]:
print("Hindi Vocabulary Size:", Hindi_vocab.vocab_size)
print("Hindi Word to Index Mapping:", Hindi_vocab.word2idx)
print("Hindi Index to Word Mapping:", Hindi_vocab.idx2word)


Hindi Vocabulary Size: 1685
Hindi Word to Index Mapping: {'<pad>': 0, '<sos>': 1, '<eos>': 2, 'अपने': 3, 'अनुप्रयोग': 4, 'को': 5, 'पहुंचनीयता': 6, 'व्यायाम': 7, 'का': 8, 'लाभ': 9, 'दें': 10, 'एक्सेर्साइसर': 11, 'अन्वेषक': 12, 'निचले': 13, 'पटल': 14, 'के': 15, 'लिए': 16, 'डिफोल्ट': 17, 'प्लगइन': 18, 'खाका': 19, 'ऊपरी': 20, 'उन': 21, 'प्लगइनों': 22, 'की': 23, 'सूची': 24, 'जिन्हें': 25, 'रूप': 26, 'से': 27, 'निष्क्रिय': 28, 'किया': 29, 'गया': 30, 'है': 31, 'अवधि': 32, 'हाइलाइट': 33, 'रकें': 34, 'पहुंचनीय': 35, 'आसंधि': 36, 'नोड': 37, 'चुनते': 38, 'समय': 39, 'बक्से': 40, 'सीमांत': 41, 'बोर्डर': 42, 'रंग': 43, 'करें': 44, 'किए': 45, 'गए': 46, 'और': 47, 'अपारदर्शिता।': 48, 'भराई': 49, 'पारदर्शिता।': 50, 'एपीआई': 51, 'विचरक': 52, 'इस': 53, 'जिसे': 54, 'प्राप्त': 55, 'हो': 56, 'उसकी': 57, 'विभिन्न': 58, 'विधियों': 59, 'मेथड': 60, 'में': 61, 'विचरण': 62, 'निजी': 63, 'गुणों': 64, 'छिपाएं': 65, 'विधि': 66, 'गुणधर्म': 67, 'मान': 68, 'आईपाइथन': 69, 'कन्सोल': 70, 'चुने': 71, 'एक्सेसेबेल': 72, 'काम':

In [20]:
print("English Vocabulary Size:", English_vocab.vocab_size)
print("English Word to Index Mapping:", English_vocab.word2idx)
print("English Index to Word Mapping:", English_vocab.idx2word)

English Vocabulary Size: 1511
English Word to Index Mapping: {'<pad>': 0, 'give': 1, 'your': 2, 'application': 3, 'an': 4, 'accessibility': 5, 'workout': 6, '<eos>': 7, 'accerciser': 8, 'explorer': 9, 'the': 10, 'default': 11, 'plugin': 12, 'layout': 13, 'for': 14, 'bottom': 15, 'panel': 16, 'top': 17, 'a': 18, 'list': 19, 'of': 20, 'plugins': 21, 'that': 22, 'are': 23, 'disabled': 24, 'by': 25, 'highlight': 26, 'duration': 27, 'box': 28, 'when': 29, 'selecting': 30, 'accessible': 31, 'nodes': 32, 'border': 33, 'color': 34, 'and': 35, 'opacity': 36, 'fill': 37, 'api': 38, 'browser': 39, 'browse': 40, 'various': 41, 'methods': 42, 'current': 43, 'hide': 44, 'private': 45, 'attributes': 46, 'method': 47, 'property': 48, 'value': 49, 'ipython': 50, 'console': 51, 'interactive': 52, 'manipulating': 53, 'currently': 54, 'selected': 55, 'event': 56, 'monitor': 57, 'events': 58, 'c': 59, 'lear': 60, 'selection': 61, 'everything': 62, 'source': 63, 'shows': 64, 'as': 65, 'they': 66, 'occur': 6

In [21]:
from torch.nn.utils.rnn import pad_sequence

class parallelData(Dataset):
    def __init__(self):
        self.x = Hindi_vocab.x
        self.y = English_vocab.x

    def __getitem__(self,i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return len(self.x)

def collate_fn(batch):
    x_batch, y_batch = zip(*batch)
    # No need for padding since it's already handled in Vocab
    x_batch = torch.stack(x_batch, dim=0)  # Stack already padded tensors
    y_batch = torch.stack(y_batch, dim=0)
    return x_batch, y_batch




In [22]:
# Check the original sentence data
print("Original Data Hindi Sentence:", data[0][0])

# Instantiate parallelData
dataset = parallelData()
print(dataset[0])
print(len(dataset))

# Retrieve the indices tensor for the first sample
indices_tensor = dataset[0][0]

# Convert indices tensor to a list of indices
indices = indices_tensor.tolist()

# Map indices to words using idx2word
words = [Hindi_vocab.idx2word.get(idx, '<unknown>') for idx in indices]

# Print the resulting words
print("Converted Words from Dataset Indices:")
print(' '.join(words))



Original Data Hindi Sentence: ['<sos>', 'अपने', 'अनुप्रयोग', 'को', 'पहुंचनीयता', 'व्यायाम', 'का', 'लाभ', 'दें', '<eos>']
(tensor([ 1,  3,  4,  5,  6,  7,  8,  9, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0]), tensor([1, 2, 3, 4, 5, 6, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
10000
Converted Words from Dataset Indices:
<sos> अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें <eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [23]:
# Original data
print("Original Data Hindi Sentence:", data[0][0])

# Data from parallelData class
print("Dataset Hindi Sentence:", dataset[0][0])


Original Data Hindi Sentence: ['<sos>', 'अपने', 'अनुप्रयोग', 'को', 'पहुंचनीयता', 'व्यायाम', 'का', 'लाभ', 'दें', '<eos>']
Dataset Hindi Sentence: tensor([ 1,  3,  4,  5,  6,  7,  8,  9, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0])


In [24]:
a = dataset[0][1].shape[0]
b = dataset[0][0].shape[0]
for i in range(len(dataset)):
  if a != dataset[i][1].shape[0] or b != dataset[i][0].shape[0]:
    print(a,dataset[i][0].shape[0],b,dataset[i][1].shape[0])

In [25]:
#Model preparation

class encoder(nn.Module):

  def __init__(self, input_size, embedding_size, hidden_size, layers):
    '''
    input_size = size of vocab
    embedding_size = embedding dim
    hidden_size = hidden state size
    layer = num of layers of lstms
    '''
    super().__init__()
    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)

  def forward(self,x):
    '''
    x shape = [batch_size, sentence]
    one complete sentence represents a "sequence"
    '''
    x = self.embed(x) # shape [batch_size,  sentence, embed_size]
    output, (hidden_state, cell_state) = self.lstm(x) #shape output =[batch_size, seq_len, hidden_size] , hidden_state = [layers, batch_size, hidden_size], cell_state = [layers, batch_size, hidden_size]

    return hidden_state, cell_state







class decoder(nn.Module):

  def __init__(self,input_size, embedding_size, hidden_size, layers):
    '''
    same configuration as encoder
    here input_size = size of hindi vocab
    '''
    super().__init__()
    self.embed = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size) # output size = (*,embedding_size)
    self.lstm = nn.LSTM(input_size=embedding_size, hidden_size= hidden_size, num_layers=layers, batch_first = True)
    self.fc = nn.Linear(in_features=hidden_size, out_features=input_size) #since output would be prob distribution among hindi vocab therefore out_feature=input_size

  def forward(self,x,hidden_state, cell_state):
    '''
    to have control over output we have to take sentence as word by word
    therefore seq_len would be 1 as input is  one word not the whole sentence
    x = [batch_size] ->required-> [batch_size, 1] (1 is seq_len)
    '''
    # print(x.shape)
    x = x.reshape(-1,1) # shape [batch, 1]
    # print(x.shape)
    x = self.embed(x) # shape [batch, 1, embed_dim]

    output, (hidden_state, cell_state) = self.lstm(x, (hidden_state, cell_state)) # shape output=>[batch, 1, hidden_size], hidden=>[layers, batch, hidden_size]
    
    output = self.fc(output) # shape [batch, 1, hindi_vocab_size]
    
    #just for removing extra dim
    output = output.squeeze(dim=1) #shape [batch, hindi_vocab_size]

    return output, hidden_state, cell_state



class seq2seq(nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder


  def forward(self, input, target, teaching_force=0.5):
    '''
    input = batch of english sentences[batch, sentece(padded)]
    target = batch of hindi sentences [batch, sentence(padded)] 
    '''
    batch_size = input.shape[0]   #4
    seq_len = target.shape[1]    #14
    hindi_vocab_size = Hindi_vocab.vocab_size   #50

    output = torch.zeros((seq_len, batch_size, hindi_vocab_size)).to(device)   #14, 4, 50

    hidden, cell = self.encoder(input)
    target = target.permute(1,0) # shape [seq, batch]   4x14 is changed to 14x4
    x = target[0] #  token shape= [batch] it contains index of <sos> token for each sentence in batch

    for i in range(1, seq_len):
      out, hidden, cell = self.decoder(x, hidden, cell) #out shape = [batch, vocab_size]
      output[i] = out
      decoder_guess = out.argmax(1)# taking the word with max value(confidence)  shape = [batch of words]

      if random.random() < teaching_force:
        x = target[i]
      else:
        x =  decoder_guess
    
    return output  #shape[seq_len, batch_size, vocab_size]

In [26]:
##training
epochs = 100
learning_rate = 0.001
batch_size = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
embedding_size = 300
hidden_size = 1024
layers = 2

In [27]:
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
print("No of batches:", len(loader))

for batch in loader:
    x_batch, y_batch = batch
    print("Shape of x_batch:", x_batch.shape)
    print("Shape of y_batch:", y_batch.shape)
    break  # Only need to check the shape of one batch



No of batches: 100
Shape of x_batch: torch.Size([100, 40])
Shape of y_batch: torch.Size([100, 42])


In [28]:
it = iter(loader)
x,y = next(it)  #x is hindi y is english
print(x.shape,y.shape)

torch.Size([100, 40]) torch.Size([100, 42])


In [29]:
ENC = encoder(English_vocab.vocab_size, embedding_size, hidden_size, layers).to(device)
DE = decoder(Hindi_vocab.vocab_size, embedding_size, hidden_size, layers).to(device)
model = seq2seq(ENC,DE).to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=0)

In [30]:
print(f"English vocab size: {English_vocab.vocab_size}")
print(f"Hindi vocab size: {Hindi_vocab.vocab_size}")


English vocab size: 1511
Hindi vocab size: 1685


In [31]:

for epoch in range(epochs):
  for id,(x,y) in (enumerate(tqdm(loader))):
    x = x.long().to(device)
    y = y.long().to(device)#[batch,seq]

    output = model(y,x)# [seq, batch, vocab]   y is input and x is target
    output = output[1:].reshape(-1,output.shape[2])
    x = x.permute(1,0)#[seq, batch]
    x = x[1:].reshape(-1)

    optimizer.zero_grad()
    loss = criterion(output,x)

    loss.backward()
    optimizer.step()

    if id%20 == 0:
      print(f'[{epoch+1}/{epochs}] loss=>{loss.item()}')

  0%|          | 0/100 [00:00<?, ?it/s]

[1/100] loss=>7.425464153289795
[1/100] loss=>5.107882499694824
[1/100] loss=>4.922605037689209
[1/100] loss=>4.538670539855957
[1/100] loss=>4.701674461364746


  0%|          | 0/100 [00:00<?, ?it/s]

[2/100] loss=>4.143955230712891
[2/100] loss=>3.983741521835327
[2/100] loss=>4.569690227508545
[2/100] loss=>4.364144325256348
[2/100] loss=>3.6261379718780518


  0%|          | 0/100 [00:00<?, ?it/s]

...

[98/100] loss=>0.0596219040453434
[98/100] loss=>0.07228881120681763
[98/100] loss=>0.09684088826179504
[98/100] loss=>0.10435336083173752
[98/100] loss=>0.2671838700771332


  0%|          | 0/100 [00:00<?, ?it/s]

[99/100] loss=>0.08102011680603027
[99/100] loss=>0.0698753148317337
[99/100] loss=>0.10945463180541992
[99/100] loss=>0.10351884365081787
[99/100] loss=>0.18097151815891266


  0%|          | 0/100 [00:00<?, ?it/s]

[100/100] loss=>0.07456283271312714
[100/100] loss=>0.14994290471076965
[100/100] loss=>0.06539963185787201
[100/100] loss=>0.1171174943447113
[100/100] loss=>0.09428831189870834


In [32]:
def predict(x):
    for idx in x:
      if idx == 0:
        break
      print(English_vocab.idx2word[int(idx)],end=' ')     # shape= 11
    
    print()

    x = x.long().reshape(1,-1).to(device)   # shape = [1, 11]
    ans = translate(x)
    for id in ans:
      print(Hindi_vocab.idx2word[id],end=' ');

    print()
    

In [33]:
def translate(input):
      #input = batch of english sentences[batch, sentece(padded)]
      with torch.no_grad():
        guess = []
        hidden, cell = model.encoder(input)
        # x = torch.ones((1)).float().to(device) #  token
        x = torch.ones((1)).long().to(device)
        while True:
          out, hidden, cell = model.decoder(x, hidden, cell) #out shape = [batch, vocab_size]
          x = out.argmax(1)# taking the word with max value(confidence)  shape = [batch of words]
          guess.append(int(x[0].detach().cpu()))

          if x == 2:
            break

      return guess
     

In [34]:
len(dataset)

10000

In [35]:
t=1
print("original hindi:", data[t][0])
predict(dataset[t][1])

original hindi: ['<sos>', 'एक्सेर्साइसर', 'पहुंचनीयता', 'अन्वेषक', '<eos>']
accerciser accessibility explorer <eos> 
एक्सेर्साइसर पहुंचनीयता अन्वेषक <eos> 


In [36]:
df = pd.read_csv('/home/sudhanshu/Desktop/resume projects/eng to hindi/my machine translator/hindi_english_parallel.csv')
df.head()

,hindi,english
0,अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें,Give your application an accessibility workout
1,एक्सेर्साइसर पहुंचनीयता अन्वेषक,Accerciser Accessibility Explorer
2,निचले पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the bottom panel
3,ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका,The default plugin layout for the top panel
4,उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से नि...,A list of plugins that are disabled by default


In [37]:
df2 = df.iloc[10001:10050,:]
df2.head()

,hindi,english
10001,नमूना क्रिया,Implementation
10002,क़िस्म,Type
10003,नाम,Name
10004,आर्गुमेंट्स,Arguments
10005,क़िस्म,GType


In [38]:
data2=df2.values
print(data2[0])
df2 = df2[['hindi', 'english']].copy()
df2['english'] = df2['english'].apply(cleanerEng)
df2['hindi'] = df2['hindi'].apply(cleanerHindi)


['नमूना क्रिया' 'Implementation']


In [39]:
df2['english'] = df2['english'].apply(func=cleanerEng)
df2['hindi']= df2['hindi'].apply(func= cleanerHindi)
df2['english']  = df2['english'].apply(func= lambda x : (str(x).split()))
df2['hindi'] = df2['hindi'].apply(func= lambda x : (str(x).split()))

In [40]:
df2.head(10)

,hindi,english
10001,"[नमूना, क्रिया]",[implementation]
10002,[क़िस्म],[type]
10003,[नाम],[name]
10004,[आर्गुमेंट्स],[arguments]
10005,[क़िस्म],[gtype]
10006,[पैरामीटर्सः],[paramspec]
10007,"[दाखिल, हो, रहा, हैः]","[args, s]"
10008,[अद्यतन],[update]
10009,[स्थानीय],[local]
10010,[विकल्प],"[cvs, options]"


In [41]:
df2['english'] = df2['english'].apply(func= addTokens,start=False)
df2['hindi']= df2['hindi'].apply(func= addTokens,start=True)

In [42]:
data2=df2.values
data2[0]
len(data2)

49

In [43]:
# Check the original sentence data
print("Original Data Hindi Sentence:", data2[0][0])

# Example Usage
English_vocab = Vocab(data2[:, 1], token=False)  # English source language
Hindi_vocab = Vocab(data2[:, 0], token=True)  # Hindi target language

# Instantiate parallelData
dataset2 = parallelData()

# Retrieve the indices tensor for the first sample
indices_tensor2 = dataset2[0][0]

# Convert indices tensor to a list of indices
indices2 = indices_tensor2.tolist()

# Map indices to words using idx2word
words = [Hindi_vocab.idx2word.get(idx, '<unknown>') for idx in indices2]

# Print the resulting words
print("Converted Words from Dataset Indices:")
print(' '.join(words))

Original Data Hindi Sentence: ['<sos>', 'नमूना', 'क्रिया', '<eos>']
Converted Words from Dataset Indices:
<sos> नमूना क्रिया <eos> <pad> <pad>


In [44]:
print(len(dataset2))
print(dataset2[45][1])

49
tensor([45,  2,  0,  0])


In [65]:
# Loop through the dataset and print the sentences from i = 0 to 40
for i in range(5,10):  # Adjust the range as neede
    predicted_hindi_sentence = predict(dataset[i][1])  # Assuming `pred` is the function generating predicted sentence


attach to paramspec 
बिन्दु क़िस्म मान स्थान <eos> 
update to repository update attach process breakpoints value location memory paramspec 
मेमोरी वर्णनः हाँ क़िस्म डिबगर पता मान अक्षम विक्रेता बिन्दु <eos> 
attach description yes paramspec 
शक्तिशाली डिबगिंग विकल्प औजार क़िस्म मान ब्रेकपाइन्ट <eos> 
update yes debugger address repository update attach description paramspec 
मान टॉगल सक्षम शक्तिशाली पैरामीटर्सः औजार स्थिति परिवर्तनीय <eos> 
attach disable yes paramspec 
विसम्मेलन विकल्प औजार क़िस्म मान ब्रेकपाइन्ट <eos> 
